In [1]:
import numpy as np
import pandas as pd
import math
from sklearn import metrics

from scipy.stats import entropy

import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import matplotlib.dates as mdates #to format dates on our plots
%matplotlib inline
import seaborn as sns

In [2]:
df = pd.read_csv('curr_access.csv', index_col=0)
df = df.set_index('timestamp')
df.head()

,date,time,request_url,user_id,cohort,ip,course
timestamp,,,,,,,
2018-01-26 09:55:03,2018-01-26,09:55:03,/,1,8.0,98.106.20.62,Unknown
2018-01-26 09:56:02,2018-01-26,09:56:02,java-ii,1,8.0,98.106.20.62,Web Dev
2018-01-26 09:56:06,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,98.106.20.62,Web Dev
2018-01-26 09:56:24,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,98.106.20.62,Web Dev
2018-01-26 09:56:41,2018-01-26,09:56:41,javascript-i/loops,2,22.0,98.106.20.62,Web Dev


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 543594 entries, 2018-01-26 09:55:03 to 2020-05-15 15:06:05
Data columns (total 7 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   date         543594 non-null  object 
 1   time         543594 non-null  object 
 2   request_url  543594 non-null  object 
 3   user_id      543594 non-null  int64  
 4   cohort       515738 non-null  float64
 5   ip           543594 non-null  object 
 6   course       543594 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 33.2+ MB


In [4]:
# df.drop(columns=(['date','time']), inplace=True)

In [5]:
df.head()

,date,time,request_url,user_id,cohort,ip,course
timestamp,,,,,,,
2018-01-26 09:55:03,2018-01-26,09:55:03,/,1,8.0,98.106.20.62,Unknown
2018-01-26 09:56:02,2018-01-26,09:56:02,java-ii,1,8.0,98.106.20.62,Web Dev
2018-01-26 09:56:06,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,98.106.20.62,Web Dev
2018-01-26 09:56:24,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,98.106.20.62,Web Dev
2018-01-26 09:56:41,2018-01-26,09:56:41,javascript-i/loops,2,22.0,98.106.20.62,Web Dev


In [6]:
df.request_url.value_counts()

/                                          28920
javascript-i                               10950
search/search_index.json                   10859
toc                                         9839
html-css                                    8202
                                           ...  
About_NLP                                      1
file.dat                                       1
src                                            1
professional-development/t-block-resume        1
7_exercises                                    1
Name: request_url, Length: 1811, dtype: int64

In [7]:
df.request_url.isna().sum()

0

In [8]:
df[df.cohort.isna()]

,date,time,request_url,user_id,cohort,ip,course
timestamp,,,,,,,
2018-01-26 16:46:16,2018-01-26,16:46:16,/,48,NaN,98.106.20.62,Unknown
2018-01-26 16:46:24,2018-01-26,16:46:24,spring/extra-features/form-validation,48,NaN,98.106.20.62,Web Dev
2018-01-26 17:54:24,2018-01-26,17:54:24,/,48,NaN,98.106.20.62,Unknown
2018-01-26 18:32:03,2018-01-26,18:32:03,/,48,NaN,98.106.20.62,Unknown
2018-01-26 18:32:17,2018-01-26,18:32:17,mysql/relationships/joins,48,NaN,98.106.20.62,Both
...,...,...,...,...,...,...,...
2020-05-13 16:30:22,2020-05-13,16:30:22,8-clustering/project,349,NaN,157.81.5.3,Unknown
2020-05-13 16:30:38,2020-05-13,16:30:38,8-clustering/5-model,349,NaN,157.81.5.3,Unknown
2020-05-13 17:36:13,2020-05-13,17:36:13,/,366,NaN,100.89.63.138,Unknown
